In [11]:
# Import libraries
import numpy as np
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

Dataset: Bee or Wasp? from Kaggle

## Data preparation

In [2]:
# set directory for train and test data
train_ds = './data/train/'
test_ds = './data/test/'

In [3]:
# Rescalling the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen
train_generator = train_datagen.flow_from_directory(
        train_ds,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True)

# Flow test images in batches using train_datagen
test_generator = test_datagen.flow_from_directory(
        test_ds,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [4]:
train_generator.class_indices

{'bee': 0, 'wasp': 1}

In [5]:
test_generator.class_indices

{'bee': 0, 'wasp': 1}

## Building CNN Model

In [12]:
# Add custom layers
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

2023-11-22 21:29:57.737458: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1. **Sequential Model**:
   - The `Sequential` model is a linear stack of layers in Keras. It allows you to create models layer-by-layer in a step-by-step fashion.

2. **Convolutional Layer (Conv2D)**:
   - `Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))`:
     - This is the first layer of your network and a convolutional layer.
     - It has 32 filters (or kernels), each of size 3x3. Filters are used to extract features from the input image.
     - `activation='relu'` indicates that the Rectified Linear Unit (ReLU) function is used as an activation function. ReLU introduces non-linearity to the model, allowing it to learn more complex patterns.
     - `input_shape=(150, 150, 3)` defines the shape of the input data: 150x150 pixels and 3 channels (assuming color images, typically RGB).

3. **Max Pooling Layer (MaxPooling2D)**:
   - `MaxPooling2D(2, 2)`:
     - This layer reduces the spatial dimensions (width and height) of the input volume.
     - It performs down-sampling by dividing the input into pools of size 2x2 and taking the maximum value of each pool. This helps reduce the number of parameters and computation in the network, and also controls overfitting.

4. **Flattening (Flatten)**:
   - `Flatten()`:
     - This layer flattens the 2D arrays from the previous layers into a 1D vector. This step is necessary because the Dense layers expect 1D inputs.

5. **Fully Connected Layer (Dense)**:
   - `Dense(64, activation='relu')`:
     - This is a fully connected layer with 64 neurons.
     - It takes the flattened input and applies weights, biases, and the ReLU activation function. This layer allows the network to learn non-linear combinations of the high-level features extracted by the convolutional layers.

6. **Output Layer (Dense)**:
   - `Dense(1, activation='sigmoid')`:
     - This is the output layer of the model with a single neuron.
     - Since this is a binary classification problem (bee or wasp), one neuron is sufficient for output. 
     - The `sigmoid` activation function is used, which outputs a value between 0 and 1, representing the probability of one class (e.g., a bee). A value close to 1 indicates a high probability of the class, while a value close to 0 indicates a low probability.

In summary, your model starts with a convolutional layer to extract features from the image, followed by max pooling to reduce dimensionality. Then, it flattens the output and uses dense layers to further process the data. The final output is obtained through a sigmoid activation function in the last dense layer, suitable for binary classification.

In [13]:
# Model compilation
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.002, momentum=0.8),
              metrics=['accuracy'])

In [14]:
# Question 2: Number of Parameters in the Convolutional Layer
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

Q2 Answer: 896

## Trainning the model

In [15]:
history = model.fit(
    train_generator,
   # steps_per_epoch=10,
    epochs=10,
    validation_data=test_generator,
   # validation_steps=10
)

Epoch 1/10


2023-11-22 21:30:10.238826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - ETA: 0s - loss: 0.6512 - accuracy: 0.6089

2023-11-22 21:30:38.512475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - 31s 167ms/step - loss: 0.6512 - accuracy: 0.6089 - val_loss: 0.5874 - val_accuracy: 0.6874
Epoch 2/10
184/184 [==============================] - 33s 181ms/step - loss: 0.5634 - accuracy: 0.7068 - val_loss: 0.5818 - val_accuracy: 0.6765
Epoch 3/10
184/184 [==============================] - 31s 170ms/step - loss: 0.5189 - accuracy: 0.7514 - val_loss: 0.5422 - val_accuracy: 0.7331
Epoch 4/10
184/184 [==============================] - 31s 169ms/step - loss: 0.4969 - accuracy: 0.7637 - val_loss: 0.5827 - val_accuracy: 0.6972
Epoch 5/10
184/184 [==============================] - 31s 166ms/step - loss: 0.4631 - accuracy: 0.7917 - val_loss: 0.5387 - val_accuracy: 0.7462
Epoch 6/10
184/184 [==============================] - 32s 176ms/step - loss: 0.4485 - accuracy: 0.8045 - val_loss: 0.5036 - val_accuracy: 0.7647
Epoch 7/10
184/184 [==============================] - 32s 173ms/step - loss: 0.4191 - accuracy: 0.8213 - val_loss: 0.5032 - val_accuracy: 0.7

In [ ]:
# Question 3: What is the median of training accuracy for all the epochs for this model?
accuracies = history.history['accuracy']
median_accuracy = np.median(accuracies).round(3)
print(median_accuracy)

0.728


In [ ]:
history.history

{'loss': [0.6852636337280273,
  0.6673799157142639,
  0.6404879689216614,
  0.6094634532928467,
  0.5739327669143677,
  0.5284214019775391,
  0.49343734979629517,
  0.47613582015037537,
  0.44612228870391846,
  0.43343669176101685],
 'accuracy': [0.5499048233032227,
  0.5722056031227112,
  0.6293174028396606,
  0.6619526743888855,
  0.7068262100219727,
  0.7489801645278931,
  0.775088369846344,
  0.7916780114173889,
  0.8052760362625122,
  0.8011966347694397],
 'val_loss': [0.6636930704116821,
  0.6285654902458191,
  0.6288901567459106,
  0.5801663398742676,
  0.5542347431182861,
  0.5648717880249023,
  0.5618477463722229,
  0.5200570821762085,
  0.5260006189346313,
  0.5290839672088623],
 'val_accuracy': [0.5446622967720032,
  0.6405228972434998,
  0.5816993713378906,
  0.6928104758262634,
  0.7200435996055603,
  0.7004357576370239,
  0.7167755961418152,
  0.7527233362197876,
  0.7320261597633362,
  0.7527233362197876]}

In [ ]:
# Question 4: What is the standard deviation of training loss for all the epochs for this model?
losses = history.history['loss']
std_loss = np.std(losses).round(3)
print(std_loss)

0.088


## Data Augmentation

In [ ]:
# Data generator with augmentations for the training set
train_datagen_augmented = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for the test set (without augmentations)
test_datagen = ImageDataGenerator()

# Create augmented training and regular test generators
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_ds,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_ds,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [ ]:
# Continue training the model for 10 more epochs
history = model.fit(
    train_generator_augmented,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
184/184 [==============================] - 23s 123ms/step - loss: 57.1059 - accuracy: 0.5102 - val_loss: 0.6904 - val_accuracy: 0.5370
Epoch 2/10
184/184 [==============================] - 23s 123ms/step - loss: 0.6915 - accuracy: 0.5377 - val_loss: 0.6926 - val_accuracy: 0.5370
Epoch 3/10
184/184 [==============================] - 23s 124ms/step - loss: 0.6917 - accuracy: 0.5374 - val_loss: 0.6911 - val_accuracy: 0.5370
Epoch 4/10
184/184 [==============================] - 23s 123ms/step - loss: 0.6908 - accuracy: 0.5374 - val_loss: 0.6906 - val_accuracy: 0.5370
Epoch 5/10
184/184 [==============================] - 23s 125ms/step - loss: 0.6909 - accuracy: 0.5374 - val_loss: 0.6905 - val_accuracy: 0.5370
Epoch 6/10
184/184 [==============================] - 23s 123ms/step - loss: 0.6905 - accuracy: 0.5374 - val_loss: 0.6905 - val_accuracy: 0.5370
Epoch 7/10
184/184 [==============================] - 23s 124ms/step - loss: 0.6904 - accuracy: 0.5374 - val_loss: 0.6904 - val_a

In [ ]:
# Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?
test_losses = history.history['val_loss']
mean_test_loss = np.mean(accuracies).round(3)
print(mean_test_loss)

0.704


In [ ]:
# Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
test_accuracies = history.history['val_accuracy']
last_5_accuracies = test_accuracies[5:10]
mean_last_5_acc = np.mean(last_5_accuracies).round(3)
print(mean_last_5_acc)

0.537
